# Langchain Introduction

Installing necessary libraries

In [ ]:
!pip install langchain==0.0.208 deeplake openai tiktoken

## LLMs

In [11]:
from langchain.llms import OpenAI
from getpass import getpass
import os

In [10]:
OPENAI_API_KEY = getpass("Enter the OpenAI API key")

Enter the OpenAI API key··········


In [12]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [18]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY,model="text-davinci-003", temperature=0.9)

In [19]:
text = "Suggest a personalized diet plan for a vegetarian person who's trying to lose fat and build muscles and can eat dairy products. Add more Indian food the person and likes spicy food."


In [20]:
print(llm(text))



Breakfast:

-1 cup cooked oats with 1/2 cup low fat milk, 1/2 cup mashed banana, 1 tablespoon honey, 1 tablespoon raisins, and a sprinkle of cinnamon. 

-1 cup sprouted moong dal parantha with a low fat paneer and vegetable stuffing, served with 1/2 cup skimmed milk yoghurt.

Snack 1: 
-1 cup air-popped popcorn sprinkled with 1/2 teaspoon of olive oil, 1 tablespoon of nutritional yeast, and a pinch of turmeric. 

Lunch:
-1 cup sprouted green gram and millet biryani accompanied with 1/2 cup cucumber raita and 1 tablespoon of olive oil. 

Snack 2:
-1 serving of Kale and Spinach Idli with peanut chutney.

Dinner: 
-1 cup of dal makhani with 1 cup of steamed brown rice and 1 cup of roasted bell peppers and onions. 

Snack 3:
-1 whole wheat bhatura with a side of low fat paneer and vegetable curry.


## Chains

In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [24]:
prompt = PromptTemplate(
    input_variables = ["product"],
    template = "What is a good name for a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
print(chain.run("cricket bats made of bamboo wood"))



Bamboo Bash Cricket Company


## Memory

In [26]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [27]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [30]:
conversation.predict(input="Tell me about yourself.")
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with machine learning?")

print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! My name is AI-bot and I'm an artificial intelligence robot. I was designed to interact with humans in a conversational manner. I'm programmed to understand common language and respond with relevant information. I'm also able to learn from conversations and store new information in my memory. I'm still learning about the world and more every day.
Human: What can you do?
AI:  I can help answer questions, provide information, and even assist in tasks. I'm able to recognize voice commands, and I can even remember our conversations, which allows me to provide a more personalized experience. I'm always learning, so as I grow I'll be able to do mo

## Vector Store - DeepLake

In [31]:
DEEPLAKE_API_KEY = getpass("Enter deeplake API key")

Enter deeplake API key··········


In [32]:
os.environ["DEEPLAKE_API_KEY"] = DEEPLAKE_API_KEY

In [33]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [35]:
llm = OpenAI(model="text-davinci-003", temperature=0, openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# Creating deep lake dataset

my_activeloop_org_id = "randhirsingh"
my_activeloop_dataset_name = "langchain_01_dataset"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings, token=DEEPLAKE_API_KEY)

db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


100%|██████████| 2/2 [00:00<00:00,  7.69it/s]


Dataset(path='hub://randhirsingh/langchain_01_dataset', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   


['1e703bf6-6cdd-11ee-99e1-0242ac1c000c',
 '1e703d4a-6cdd-11ee-99e1-0242ac1c000c']

In [36]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [37]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [38]:
tools = [
    Tool(
        name="Retrievel QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions"
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [39]:
response = agent.run("When was Napolean born")
print(response)



> Entering new  chain...
 I need to find out when Napolean was born
Action: Retrievel QA System
Action Input: When was Napolean born
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.
